### PyTorch Basics:

In [2]:
import torch
import numpy as np

Creating Tensors:

PyTorch is interesting in the sense that the mathematical operations are stored in a Dynamic Computational Graph (DCG). PyTorch runs in a "Symbolic Programming" way. Reading in commands, storing as a DCG. Computation is done when the script is run.

In [3]:
x = torch.ones(3, dtype=float)
y = 5 
w = torch.from_numpy(np.array([1,2,3])).type(torch.DoubleTensor)
w.requires_grad = True
y_hat = torch.dot(w, x) + 3 
Loss = (y-y_hat) ** 2


PyTorch uses the Backpropogation technique to compute the derivatives of a function. Usually the cost/loss function for a model is a sequence of operations applied to the model parameters. These operations are stored in a DCG as mentioned above. To compute the gradient of a cost function (L):

- Set "requires_grad" attribute of the parameter tensor to true. This tells PyTorch that we will be evaluating the derivative of a function at this point. Thus we should track the operations, we will utilise the DCG through chain rule to compute the derivative. 
- Call the "backward" method on the loss function to output the derivative of cost at these model params. 
- The gradient of the loss is stored as the "grad" attribute of the **model parameters**, not the loss function.

The code below shows a demonstration of this, we're expecting the gradient of this to be (8,8,8). 

In [4]:
Loss.backward()
w.grad

tensor([8., 8., 8.], dtype=torch.float64)

## Linear Regression Example:

In [5]:
import pandas as pd
from sklearn import datasets
import torch.nn as nn 
import numpy as np


In [29]:
X_train, y_train = datasets.fetch_california_housing(as_frame=True, return_X_y=True)
# PyTorch functions with floats. 
X_train = X_train.astype(np.float32)
y_train = y_train.astype(np.float32)
print(X_train.shape)
print(y_train.shape)

(20640, 8)
(20640,)
   MedInc  HouseAge  AveRooms  AveBedrms  Population  AveOccup   Latitude  \
0  8.3252      41.0  6.984127   1.023810       322.0  2.555556  37.880001   
1  8.3014      21.0  6.238137   0.971880      2401.0  2.109842  37.860001   
2  7.2574      52.0  8.288136   1.073446       496.0  2.802260  37.849998   
3  5.6431      52.0  5.817352   1.073059       558.0  2.547945  37.849998   
4  3.8462      52.0  6.281853   1.081081       565.0  2.181467  37.849998   

    Longitude  
0 -122.230003  
1 -122.220001  
2 -122.239998  
3 -122.250000  
4 -122.250000  


In [49]:
X_tensor = torch.nn.functional.normalize(torch.from_numpy(X_train.values))
y_tensor = torch.from_numpy(y_train.values)
y_tensor = y_tensor.view(-1, 1)
num_features = X_tensor.shape[1]
num_datapoints = X_tensor.shape[0]

In [75]:
model = nn.Linear(num_features, 1)
model_params = [t.data for t in model.parameters()]
print(model_params)

[tensor([[-0.1584,  0.0747,  0.2309,  0.2794,  0.0714, -0.0722,  0.2221, -0.1983]]), tensor([-0.3461])]


In [76]:

print(model(X_tensor).shape)

torch.Size([20640, 1])


In [79]:
num_iters = 50000
learning_rate = 0.5
loss_func = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

for epoch in range(1, num_iters+1):
    # What do we need to do in each iteration of SGD?:
    # Feed forward, Compute loss, gradient of loss, update model params.

    # Feed Forward:
    predictions = model(X_tensor)

    # Computing Loss
    loss = loss_func(y_tensor, predictions)

    # Call the backward method to compute the gradient. Recall done implicitly as model params are store in the instance. 
    loss.backward()

    # Apply a step of gradient descent
    optimizer.step()

    # Set the gradient of model params to be 0. 
    optimizer.zero_grad()

    # Printing loss every 200 iterations.
    if epoch % 5000 == 0:
        print(f'The loss for Epoch {epoch} is: {loss:.2f}')

The loss for Epoch 5000 is: 1.21
The loss for Epoch 10000 is: 1.19
The loss for Epoch 15000 is: 1.17
The loss for Epoch 20000 is: 1.15
The loss for Epoch 25000 is: 1.13
The loss for Epoch 30000 is: 1.12
The loss for Epoch 35000 is: 1.11
The loss for Epoch 40000 is: 1.10
The loss for Epoch 45000 is: 1.09
The loss for Epoch 50000 is: 1.08
